## Setup

In [2]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt
# matplotlib.backend
# matplotlib.validate_backend("GTK3Ag")

In [3]:
import numpy as np
from tqdm.auto import tqdm as progressbar
import matplotlib.pyplot as plt
import pickle
import matplotlib.animation as animation
import cv2
import cv2.aruco as aruco
import common as c
from os.path import join
from calib import calibrate_charuco_local, load_board, load_coefficients, save_coefficients

In [4]:
data_path="/home/freitas/TCC/ender-laser-scanner/pics/v3_raspi_16x16/"

In [5]:
import json

# Calib params
with open("calib.pkl", "rb") as f:
    calib = pickle.load(f)
    
mtx, dist = load_coefficients("/home/freitas/TCC/ender-laser-scanner/calib_data/calibration_charuco.yml")

board, aruco_dict = load_board("board.pkl")

print(board.getChessboardSize())
print(board.getMarkerLength())
print(board.getSquareLength())

(16, 16)
9.375
12.5


In [6]:
def load(path):
    info = {}
    data = []
    with open(join(path,"points.json")) as f:
        info = json.loads(f.read())

    for fig, point in info.items():
        image =  cv2.cvtColor(cv2.imread(join(path, fig)), cv2.COLOR_BGR2RGB)
        image = cv2.undistort(image, mtx, dist, None, mtx)
        data.append( (point, image) )

    return data


arucoParams = aruco.DetectorParameters_create()

def find_charuco_pose(image, mtx, dist):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    corners, ids, rejected = aruco.detectMarkers( gray, aruco_dict, cameraMatrix=mtx, parameters=arucoParams, distCoeff=dist)
    resp, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco( markerCorners=corners, markerIds=ids, image=gray, board=board, cameraMatrix=mtx, distCoeffs=dist)

    if(resp > 6):
        retval, rvec, tvec	=	cv2.aruco.estimatePoseCharucoBoard(charuco_corners, charuco_ids, board, mtx, dist, None,None)
    else:
        return None, None
    if not retval:
        print("erro")
    return rvec, tvec

In [7]:
mtx, dist = load_coefficients("/home/freitas/TCC/ender-laser-scanner/calib_data/calibration_charuco.yml")
print(mtx)
print(dist)

[[2.64094913e+03 0.00000000e+00 1.61864666e+03]
 [0.00000000e+00 2.64268137e+03 1.27628315e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[ 0.21625407 -0.5614533   0.00181032 -0.00367292  0.34455463]]


In [8]:
data = load(data_path+"auto_exp")
coords = []

for point, image in data:
    rvec, tvec = find_charuco_pose(image, mtx, None)
    if rvec is None or tvec is None:
        print("OOPS")
    coords.append((rvec, tvec))

In [8]:
import common as c
c.animate((g[1] for g in data))

## Funcs

In [25]:
rvecs = [d[0] for d in coords]
tvecs = [d[1] for d in coords]

points = [np.array(d[0]) for d in data]
images = [d[1] for d in data]

## Plotar Eixos coords

In [18]:
import matplotlib.pyplot as plt
import importlib
import common as co
importlib.reload(co)
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

fig, ax = co.plot3d()

for (point, image), (rvec, tvec) in zip(data, coords):
    inv = np.linalg.inv(co.vec2M(rvec, tvec))
    co.plot_axes(ax, inv)

# Chessboard
ax.scatter(board.chessboardCorners[:,0], board.chessboardCorners[:,1], board.chessboardCorners[:,2], color="k", label="Pontos de $CW$")
ax.set_xlabel("$x_{CW}$")
ax.set_ylabel("$y_{CW}$")
ax.set_zlabel("$z_{CW}$")
co.set_axes_equal(ax)

plt.title("Representação das Transformações $H_{C_i}$")
plt.legend()

# Calibração hand-eye

Usando 3 DoF, AX = XB reduzido

In [19]:
fig, ax = co.plot3d()

rvec, tvec = coords[0]

# camera->Target p/ primeira vista
cT0 = co.vec2M(rvec, tvec)
ta = []
for rvec, tvec in coords[1:]:

    # camera->target p/ vista i
    cTi = co.vec2M(rvec, tvec)

    # Transformada primeira camera -> camera i
    Ai = cT0@np.linalg.inv(cTi)

    # print(np.round(Ai[:3,:3],2)) # tem que ser I3

    # Translação primeira camera -> camera i
    ta.append(Ai[:3,3])

    # Axes to be sure
    co.plot_axes(ax, Ai)

co.set_axes_equal(ax)
plt.legend()

# Scatter dos pontos sem os axes
fig, ax = co.plot3d()
ta = np.stack(ta).T

# Pontos da impressora
P = np.stack([d[0] for d in data[1:]]).T
p0 = data[0][0]
# Tira o primeiro, pq é transformada vista 0 -> vista i
P[0] = P[0] - p0[0]
P[1] = P[1] - p0[1]
P[2] = P[2] - p0[2]

ax.scatter(ta[0], ta[1], ta[2])
ax.scatter(P[0], P[1], P[2])

No handles with labels found to put in legend.


## Estimar affine

In [31]:
R, scale = cv2.estimateAffine3D(P.T, ta.T,force_rotation=True)
print(R,scale)
tal = R[:,:3]@P #+ R[:,3]
tal[0] = tal[0] + R[0,3]
tal[1] = tal[1] + R[1,3]
tal[2] = tal[2] + R[2,3]

fig, ax = co.plot3d()
ax.scatter(ta[0], ta[1], ta[2])
ax.scatter(tal[0], tal[1], tal[2])

co.set_axes_equal(ax)
np.save("/home/freitas/TCC/ender-laser-scanner/calib_data/cam_to_printer", R)
np.savetxt("tex/cam_to_printer.txt", R, delimiter=' & ', fmt='%.4f', newline=' \\\\\n')

[[ 0.99849511 -0.02474026  0.04894311 -0.01569438]
 [ 0.00430178 -0.85437493 -0.51963927  0.08782534]
 [ 0.05467178  0.51906782 -0.85298276  0.38508257]] 1.0048117570810848


# Usando `cv2.calibrateHandEye`

In [ ]:
R_target2cam = []
T_target2cam = []
for rvec, tvec in coords:

    M = np.linalg.inv(co.vec2M(rvec, tvec))
    R_target2cam.append(M[:3,:3])
    T_target2cam.append(M[:3,3])

R_gripper2base = [np.eye(3) for i in R_target2cam]
T_gripper2base = [-np.array(d[0]) for d in data]

cv2.calibrateHandEye(R_gripper2base, T_gripper2base, R_target2cam, T_target2cam)

# Validar

In [22]:
print(R)

[[ 0.99849511 -0.02474026  0.04894311 -0.01569438]
 [ 0.00430178 -0.85437493 -0.51963927  0.08782534]
 [ 0.05467178  0.51906782 -0.85298276  0.38508257]]


In [21]:
b_corners = cv2.convertPointsToHomogeneous(board.chessboardCorners)[:,0,:].T
print(b_corners.shape)
print(b_corners)

(4, 225)
[[ 12.5  25.   37.5  50.   62.5  75.   87.5 100.  112.5 125.  137.5 150.
  162.5 175.  187.5  12.5  25.   37.5  50.   62.5  75.   87.5 100.  112.5
  125.  137.5 150.  162.5 175.  187.5  12.5  25.   37.5  50.   62.5  75.
   87.5 100.  112.5 125.  137.5 150.  162.5 175.  187.5  12.5  25.   37.5
   50.   62.5  75.   87.5 100.  112.5 125.  137.5 150.  162.5 175.  187.5
   12.5  25.   37.5  50.   62.5  75.   87.5 100.  112.5 125.  137.5 150.
  162.5 175.  187.5  12.5  25.   37.5  50.   62.5  75.   87.5 100.  112.5
  125.  137.5 150.  162.5 175.  187.5  12.5  25.   37.5  50.   62.5  75.
   87.5 100.  112.5 125.  137.5 150.  162.5 175.  187.5  12.5  25.   37.5
   50.   62.5  75.   87.5 100.  112.5 125.  137.5 150.  162.5 175.  187.5
   12.5  25.   37.5  50.   62.5  75.   87.5 100.  112.5 125.  137.5 150.
  162.5 175.  187.5  12.5  25.   37.5  50.   62.5  75.   87.5 100.  112.5
  125.  137.5 150.  162.5 175.  187.5  12.5  25.   37.5  50.   62.5  75.
   87.5 100.  112.5 125.  137.5 150

In [23]:
print((R@b_corners).shape)

(3, 225)


In [27]:
i = 1

plt.figure()

pts, jac = cv2.projectPoints(board.chessboardCorners,  rvecs[i], tvecs[i],mtx, dist)
pts = pts[:,0,:].T
plt.plot(pts[0], pts[1], "r.", markersize=12)

pts, jac = cv2.projectPoints((co.vec2M(rvecs[i],tvecs[i])@b_corners).T[:,:3], np.eye(3),np.zeros((3,1)),mtx, dist)
pts = pts[:,0,:].T
plt.plot(pts[0], pts[1], "g.")

plt.imshow(data[i][1])

In [28]:
fig, ax = co.plot3d()

i = 1
j = 2

X = np.eye(4)
X[:3,:3] = np.linalg.inv(R[:3,:3])

A = np.eye(4)
A[:3,3] = R[:3,:3]@(points[j]-points[i])

co.plot_axes(ax, np.eye(4), scale=100)
board_i = co.vec2M(rvecs[i], tvecs[i])@b_corners

board_j = co.vec2M(rvecs[j], tvecs[j])@b_corners
board_j_from_i = np.linalg.inv(A)@board_i

board_i = X@board_i
board_j = X@board_j
board_j_from_i = X@board_j_from_i
ax.scatter(board_i[0], board_i[1], board_i[2])
ax.scatter(board_j[0], board_j[1], board_j[2])
ax.scatter(board_j_from_i[0], board_j_from_i[1], board_j_from_i[2])
ax.invert_zaxis()

co.set_axes_equal(ax)

In [29]:
i = 1
j = 2

A = np.eye(4)
A[:3,3] = R[:3,:3]@(points[j]-points[i])

X = np.eye(4)
X[:3,:3] = R[:3,:3]

B = np.eye(4)
B[:3,3] = points[j] - points[i]

print(A@X)
print(X@B)
print((A@X) - (X@B))
print((X@B@np.linalg.inv(X)) - (A))

[[ 9.98495113e-01 -2.47402597e-02  4.89431148e-02  5.49172312e+01]
 [ 4.30178077e-03 -8.54374929e-01 -5.19639275e-01  2.36597942e-01]
 [ 5.46717808e-02  5.19067819e-01 -8.52982764e-01  3.00694794e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.98495113e-01 -2.47402597e-02  4.89431148e-02  5.49172312e+01]
 [ 4.30178077e-03 -8.54374929e-01 -5.19639275e-01  2.36597942e-01]
 [ 5.46717808e-02  5.19067819e-01 -8.52982764e-01  3.00694794e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[ 2.22044605e-16  6.47679590e-19  1.89391665e-18  0.00000000e+00]
 [ 9.20229275e-19  0.00000000e+00 -1.36929048e-17  0.00000000e+00]
 [ 1.52365471e-17 -1.15125073e-17 -1.11022302e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
